<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Student Name:</b> Rosalie Millner</h4>
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [1]:
!nvidia-smi

Tue Oct 29 15:42:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00


In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [4]:
!pip install transformers==4.45.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 110.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.0
    Uninstalling transformers-4.46.0:
      Successfully uninstalled transformers-4.46.0


In [5]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import ( #Parameter-Efficient Fine-Tuning, goal here
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [6]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
#MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
   )# fill the gap

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        # fill the gap: get the number of trainable parameters: trainable_params
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
print_trainable_parameters(model)

trainable params: 136178560 || all params: 315119488 || trainable%: 43.21489631260127


## <b>Configuring LoRA:<b>

In [9]:
config = LoraConfig(
    r=16, #rank param #determines the size of the low-rank decomposition matrices w LoRA
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["query_key_value"],
    bias = "none",
    task_type = "QUESTION_ANSWERING" #"DIALOGUE" #"CAUSAL_LM" # fill the gap
)

model = get_peft_model(model, config) # fill the gap, using lora weights
print_trainable_parameters(model)

trainable params: 1081344 || all params: 316200832 || trainable%: 0.34198012483408013


## <b>Test the model before finetuning:<b>

In [10]:
prompt = "<human>: What equipment do I need for rock climbing?  \n<assistant>: " # # fill the gap, prompt of the format: "<human>: What equipment do I need for rock climbing?  \n <assistant>: ", with an empty response from the assistant
print(prompt)

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: What equipment do I need for rock climbing?  
<assistant>: 


**Question**: what does the temperature do in the above cell?

When generating text with a language model, the `temperature` parameter controls the randomness of the models' predictions, by indicating the extend to which the model should make predictions according to the calculated probabilities or not. One could see it as the extend to which the model is predictable or creative when generating outputs. In our case here, the `temperature` is set to 0.7 which is considered to be a low temperature ($<1.0$). Therefore, the model is rather deterministic is the sense that it samples majoritarily according to the probability distributions: it tends to select tokens with higher probabilities.  The priority of the model is therefore set to making probable choices.

In [11]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)

with torch.inference_mode(): # gradients are not needed -> faster computation
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<human>: What equipment do I need for rock climbing?  
<assistant>: 1. Rock climbing harness
<assistant>: 2. Rock climbing shoes
<assistant>: 3. Rock climbing gloves
<assistant>: 4. Rock climbing helmet
<assistant>: 5. Rock climbing board
<assistant>: 6. Rock climbing board anchor
<assistant>: 7. Rock climbing board anchor rope
<assistant>: 8. Rock climbing board anchor rope tie
<assistant>: 9. Rock climbing board anchor rope tie knot
<assistant>: 10. Rock climbing board anchor rope tie knot knot
<assistant>: 11. Rock climbing board anchor rope tie knot knot knot
<assistant>: 12. Rock climbing board anchor rope tie knot knot knot knot
<assistant>: 13. Rock climbing board anchor rope tie knot knot knot knot knot
<assistant>: 14. Rock climbing board anchor rope tie knot knot knot knot knot knot
<assistant>: 15. Rock climbing board anchor rope tie knot
CPU times: user 26.5 s, sys: 146 ms, total: 26.6 s
Wall time: 38.4 s


## <b>Loading the question/answer dataset from HuggingFace:<b>

In [12]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

README.md:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/147706 [00:00<?, ? examples/s]

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [13]:
def generate_prompt(data_point):
    # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"
    return f"<human>: {data_point['instruction']}  \n<assistant>: {data_point['demonstration']}"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)

Map:   0%|          | 0/147706 [00:00<?, ? examples/s]

In [14]:
data

Dataset({
    features: ['instruction', 'demonstration', 'meta', 'input_ids', 'attention_mask'],
    num_rows: 147706
})

## <b>Finetuning:<b>

In [15]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=400,   # try more steps if you can (originally: max_steps=200)
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.260100
2,2.089600
3,2.751500
4,2.464600
5,2.836600
6,2.597400
7,3.083000
8,2.196800
9,2.954100
10,2.644900


TrainOutput(global_step=400, training_loss=1.8987513124942779, metrics={'train_runtime': 401.7137, 'train_samples_per_second': 3.983, 'train_steps_per_second': 0.996, 'total_flos': 306705585576192.0, 'train_loss': 1.8987513124942779, 'epoch': 0.010832329086157638})

In [16]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [17]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipment do I need for rock climbing?  
<assistant>:  You will need a pair of climbing shoes, a pair of climbing gloves, a pair of climbing socks, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness

In [19]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question} \n <assistant>:" # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [20]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

- What program can I use to edit video clips I took with my phone? 

You can use the video editing software Adobe Premiere Pro, Final Cut Pro, or iMovie. These programs allow you to edit video clips, add effects, and convert them to different formats. They are all free to download and use on your computer.



- Do you know the reasons as to why people love coffee so much? 

I don’t know. I’m just a random person who loves coffee. I don’t know why people love coffee so much. I’m just a random person who loves coffee. I don’t know why people love coffee so much. I’m just a random person who loves coffee. I don’t know why people love coffee so much. I’m just a random person who loves coffee. I don’t know why people love coffee so much. I’m just a random person who loves coffee. I don’t know why people love coffee so much. I’m just a random person who loves coffee. I don’t know why people love coffee so much. I’m just a random person who loves coffee. I don’t know why people love coffee so

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [21]:
prompt_2 = "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: " # fill the gap, prompt of the format: "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: ", with an empty response from the assistant
print(prompt_2)

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>: 


In [22]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>:  I can’t taste it.  It’s too bland.  I think you need more pepper.  Can you tell me what you think?  I’m not sure what you mean by “spice” though.  Do you mean something that you want to add to the dish?  If so, I can help you with that.  Otherwise, I don’t know what you mean.  Please tell me what you think.  I’m not sure what you mean by “spice” though.  Do you mean something that you want to add to the dish?  If so, I can help you with that.  Otherwise, I don’t know what you mean.  Please tell me what you think.  I’m not sure what you mean by “spice” though.  Do you mean something that you want to add to the dish?  If so, I can help you with that.  Otherwise, I don’t know what you mean.  Please
CPU times: user 18.2 s, sys: 53.7 ms, total: 18.3 s
Wall time: 18.4 s


## Loading the preference data from Huggingface:

In [23]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

README.md:   0%|          | 0.00/171 [00:00<?, ?B/s]

dataset_final_v2.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2179 [00:00<?, ? examples/s]

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [24]:
def preprocess_data_dpo(data_point):
    # fill the gap, using dpo format
    prompt = f"<human>: {data_point['question']}\n<assistant>:" #prompt = f"<human>: {data_point['question']}\n<assistant>: {data_point['chosen']}"
    chosen_response = data_point['chosen']
    rejected_response = data_point['rejected']
    return {
        'prompt': prompt,
        'chosen': chosen_response,
        'rejected': rejected_response
    }

data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)

Map:   0%|          | 0/2179 [00:00<?, ? examples/s]

In [25]:
print(data_dpo)

Dataset({
    features: ['system', 'question', 'chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


In [26]:
data_dpo[0]

{'system': 'You are a helpful assistant',
 'question': "What are the benefits of utilizing sparse upcycling in the context of training neural networks, according to the insights provided in 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints'?",
 'chosen': 'Sparse upcycling offers several benefits in training neural networks, including improved model performance, increased efficiency, and reduced computational costs. By leveraging the knowledge contained in dense pre-trained models, sparse upcycling enables the creation of mixture-of-experts models that can achieve better accuracy and faster convergence, while also reducing the need for extensive retraining.',
 'rejected': "Sparse upcycling is not beneficial for training neural networks, as it can lead to overfitting and decreased model performance. According to 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints', sparse upcycling is only useful for reducing model size, but it does not provide any i

## Finetuning

**Question**: what is beta in dpo_args?

The `beta` parameter in DPO acts like a “temperature” for the reward signal. A lower `beta` (close to zero) makes the model more sensitive to this signal, so it prefers to generate responses that align more with the preferred samples, which is thus the case here, as we have a `beta` of value 0.1. In contrast, a higher `beta` makes the model a bit more adaptable, which is not strictly tied to preferred responses. It allows it to explore a wider range of possible outputs.

In [27]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

dpo_args = {
    "beta": 0.1,
}

trainer = DPOTrainer(
    # fill the gap
    # Data collator is not needed for DPOTrainer as it internally manages it
    model=model,
    args=training_args,
    #processing_class=tokenizer,
    beta=dpo_args["beta"],
    train_dataset=data_dpo,
    tokenizer=tokenizer,
)

trainer.processing_class = transformers.DataCollatorWithPadding(tokenizer)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:660: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:673: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:708: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/2179 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.923800
2,0.594400
3,1.234400
4,1.030400
5,1.098400
6,0.744200
7,1.003100
8,0.680900
9,0.814700
10,0.710300


TrainOutput(global_step=200, training_loss=0.1444797844128334, metrics={'train_runtime': 299.1227, 'train_samples_per_second': 2.674, 'train_steps_per_second': 0.669, 'total_flos': 0.0, 'train_loss': 0.1444797844128334, 'epoch': 0.36714089031665903})

## Test the model after DPO:

In [28]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>:  I can provide information on the taste and potential seasoning suggestions for a dish based on its ingredients and intended use. However, if you are looking for specific recommendations for adding spices, I can assist with that as well. Please let me know if you would like further assistance or if there is a particular ingredient or seasoning combination you are interested in.
CPU times: user 8.69 s, sys: 41.4 ms, total: 8.73 s
Wall time: 13 s


In [29]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question} \n <assistant>:" # construct same prompt as before
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [30]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Explain the effects of globalization on the environment."
print(generate_response(prompt))

- Do people dream in color or black and white? 

There is debate among psychologists and neurologists regarding whether people typically dream in color or black and white. Some researchers argue that color-based dreams may be more common in certain cultural or psychological contexts, while others suggest that black and white are more prevalent in non-visual dreams or those involving visual stimuli. However, it is generally accepted that some forms of dream imagery may involve a blend of color and black and white, depending on the individual's experiences and the context in which they are experiencing the dream.



- Explain the concept of economic policies in simple terms 

Economic policies are strategic decisions made by governments or other entities aimed at influencing the behavior of individuals, businesses, or the economy as a whole. They often involve a range of interventions, such as taxation, regulation, investment, and fiscal and monetary policies, to achieve specific goals s

Is the response improved after DPO?

In [31]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

For rock climbing, various equipment and skills are essential: climbing harnesses, ropes, anchor points, belay devices, crampons, belay devices, anchor points, and ropes. Additionally, personal safety gear such as helmets, gloves, and clothing may be required depending on the type of climbing and the level of experience. Personal safety is also crucial, including knowledge of first aid, emergency response, and communication skills.


Yes, the response generated by the model clearly improved after DPO: there are less repetitions, and it overall seems much more coherent and clear!

P.S. The difference in response quality is even more pronounced considering that I used 400 training steps to train the model before DPO, whereas I only used 200 training steps (which is half!) after DPO.